<a href="https://colab.research.google.com/github/JavierC11/predict_dropout_Python/blob/main/Algoritmo_Clasificacion_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd                                                                     #Pandas
import numpy as np                                                                      #Numpy
import matplotlib.pyplot as plt                                                         #Matoplotlib
from joblib import dump, load
from sklearn.preprocessing import LabelEncoder                                          #Para Volver Numero los categorias
from sklearn.model_selection import train_test_split                                    #Partir datos train, test
from sklearn.ensemble import RandomForestClassifier                                     #Crear modelo de Random Forest
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score     #Medir efectividad

In [ ]:
df = pd.read_excel('/content/Data_Proyecto.xlsx')
df

,No,Nombre Completo,Semestre,Inasistencia,Primer Parcial,Segundo Parcial,Sexo,Posible_Desercion
0,1,Ana Carolina Pérez Hernández,4,1,0.0,4.0,F,Muy Posible
1,2,José Manuel López García,4,1,10.0,7.5,M,Posible
2,3,María Isabel Torres González,4,3,8.0,8.2,F,Normal
3,4,Juan Carlos González Ramírez,4,2,15.0,9.5,M,Normal
4,5,Andrea Sofía Mendoza Aguilar,4,1,7.0,7.6,F,Normal
...,...,...,...,...,...,...,...,...
995,996,Fidelio Nodes,6,3,15.0,4.0,M,Normal
996,997,Loree Di Filippo,9,3,9.0,12.0,M,Normal
997,998,Addie Conn,1,1,13.0,9.0,M,Normal
998,999,Arlyn Laxtonne,4,3,14.0,3.0,M,Normal


In [ ]:
#Ya tenemos nuestro dataset final por lo que eliminaremos los datos no necesrios para
#el entrenamiento
df = df.drop(['Nombre Completo','No'], axis=1)

In [ ]:
#Ahora convertiemos a binario nuestra variable de sexo para que interactue mejor
#con nuestro modelo matematico, Antes de reemplzarlos eliminamos los espacios en blanco

df['Sexo'] = df['Sexo'].str.strip()

le_sexo = LabelEncoder()
df['Sexo'] = le_sexo.fit_transform(df['Sexo'])

le_posible = LabelEncoder()
df['Posible_Desercion'] = le_posible.fit_transform(df['Posible_Desercion'])


#df['Sexo'] = df['Sexo'].replace({'M':'1', 'F':'0'})
#df['Sexo']

In [ ]:
#Hombres 1
#Mujeres 0
df['Sexo'].value_counts()


Sexo
1    743
0    257
Name: count, dtype: int64

In [ ]:
#0 MUY POSIBLE
#1 NORMAL
#2 POSIBLE
df['Posible_Desercion'].value_counts()

Posible_Desercion
1    712
2    184
0    104
Name: count, dtype: int64

In [ ]:
#Partiremos el dataset entre uno de datos de prediccion y otro en la respuestas
X = df.drop('Posible_Desercion', axis=1)
y = df['Posible_Desercion']

In [ ]:
#Partimos los datos en Entrenamiento y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#Creamos el modelo de random forest con una cantidad de 100 arboles de desicion y 42 random state
#que serviran par controlar las variables aleatorias del modelo.
Model_RandomForest = RandomForestClassifier(n_estimators=100, random_state = 42)

In [ ]:
#Entrenamos o ajustamos el modelo con los datos de entrenamiento
Model_RandomForest.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
#Ahora obtenemos los resultados del modelo
y_model = Model_RandomForest.predict(X_test)

In [ ]:
#Finalmente sacamos el porcentaje de efectividad del modelo, la matriz de confusion
#y algunas metricas de clasificacion como precision, f1-score ,recall
accuracy = accuracy_score(y_test, y_model)
conf_matrix = confusion_matrix(y_test, y_model)
class_report = classification_report(y_test,y_model)

In [ ]:
#Mostramos los porcentajes
print('Efectividad: ' + str(round(accuracy*100,2)))
print()
print('Matriz Confusion: \n' + str(conf_matrix))
print()
print('Clasificaciones: \n' + str(class_report))

Efectividad: 85.33

Matriz Confusion: 
[[ 14   5   6]
 [  8 201   2]
 [ 12  11  41]]

Clasificaciones: 
              precision    recall  f1-score   support

           0       0.41      0.56      0.47        25
           1       0.93      0.95      0.94       211
           2       0.84      0.64      0.73        64

    accuracy                           0.85       300
   macro avg       0.72      0.72      0.71       300
weighted avg       0.86      0.85      0.85       300



In [ ]:
#Ahora probamos generar el resultado a un ejemplo real
example = {'Semestre':[4],'Inasistencia':[1],'Primer Parcial':[2],'Segundo Parcial':[1],'Sexo':['M']}

#Luego lo pasamos a un dataframe
student = pd.DataFrame(example)

#Pasamos a binario el campos de sexo
student['Sexo'] = le_sexo.transform(student['Sexo'])
student['Sexo']

0    1
Name: Sexo, dtype: int64

In [ ]:
#Luego ingresamos los datos el modelo
response = Model_RandomForest.predict(student)

classification_subjet = le_posible.inverse_transform(response)
print("La clasificacion del sujeto es: " +classification_subjet)

['La clasificacion del sujeto es: Muy Posible']


In [ ]:
#Ahora guardaremos nuestro modelo ya creado
dump(Model_RandomForest, 'Model_RandomForest_Students.joblib')
dump(le_sexo, 'le_sexo.joblib')
dump(le_posible, 'le_posible.joblib')

['le_posible.joblib']

In [ ]:
pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick
